# コンピューティングの使用


Azure Machine Learningジョブとしてスクリプトを実行する場合、ジョブ実行の実行コンテキストを定義する必要があります。重要な設定の1つは、スクリプトを実行するコンピュートターゲットです。これは、ローカルワークステーション（この場合はコンピュートインスタンス）、またはオンデマンドでプロビジョニングされるAzure Machine Learningマネージドコンピュートクラスターのようなリモートコンピュートターゲットになります。

このノートブックでは、コンピュートクラスタを作成し、ジョブ用のコンピュートターゲットを探索します。

## 始める前に

このノートブックのコードを実行するには、**azure-ai-ml** パッケージの最新バージョンが必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。



In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、識別子のパラメータ（サブスクリプションID、リソースグループ名、ワークスペース名）が必要です。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、ワークスペースに接続するにはデフォルト値を使用できる。

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Create a compute cluster

多くの場合、大量のデータを処理する必要がある複雑な実験や長期間の実験を処理するには、ローカルのコンピュートリソースでは不十分な場合があります。Azure Machine Learningは、さまざまなコンピュートターゲットをサポートしており、ワークスペースで定義してジョブの実行に使用できます。

Azure Machine Learning studio](https://ml.azure.com)、Python SDK、またはAzure CLIを使用して、コンピュートクラスタを作成できます。以下のコードでは、ワークスペースに `aml-cluster` という名前のコンピュートクラスタが存在するかどうかをチェックし、存在しない場合は作成します。

In [4]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "aml-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS11_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=1,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=120,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)


Creating a new cpu compute target...


コンピュートクラスタを作成した後は、以下の設定しか変更できません：

- min_instances`： 最小ノード数
- max_instances`： 最大ノード数
- idle_time_before_scale_down`: スケールダウンするまでのアイドル時間： スケールダウンまでのアイドル時間

現在、コンピュートクラスタ `aml-cluster` は最大1ノードまでしかスケールダウンできない。これを2ノードに変更して、並列計算ができるようにしてみよう。

＊エラーがでた際は再実行してください。

In [6]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

コンピュートクラスタが更新されると、その属性を表示して構成を確認することができます。

In [8]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute の名前 {cpu_cluster.name} 最大のCPU {cpu_cluster.max_instances} nodes"
    )

AMLCompute の名前 aml-cluster 最大のCPU 2 nodes


## Create a script to train a model

モデルをトレーニングするには、まず **src** フォルダに **diabetes_training.py** スクリプトを作成します。このスクリプトは学習データと同じフォルダにある**diabetes.csv**ファイルを使用します。

In [9]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

Writing src/diabetes-training.py


## Run a job on a compute cluster

これで、作成したコンピュートクラスタ上でジョブを実行する準備ができました。

> 注意
> 計算クラスタが0ノードから1ノードにスケールする必要があるためです。計算クラスタの準備ができると、スクリプトが実行されます。ジョブが終了すると、コンピュートクラスタはゼロノードまでスケールダウンします。コンピュートクラスタの状態は、**Compute**ページで確認できます。

In [10]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.52 MBs): 100%|███████

Monitor your job at https://ml.azure.com/runs/lucid_door_dswxxpm5ks?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1
